In [2]:
import os
import logging
import osmnx as ox
import numpy as np
import pandas as pd
import geopandas as gpd
import h3
import shapely
import requests
import plotly.express as px
from tqdm.notebook import tqdm
from pyproj import Transformer
from shapely.ops import linemerge
from shapely import Polygon, Point
from pyogrio.errors import DataSourceError

ox.settings.cache_folder = './data'

## functions

In [3]:
def fetch_google_places(categories:list[str], lon:float, lat:float, radius:float, apikey:str):
    
    max_results = 20

    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": apikey,
        "X-Goog-FieldMask": "places.id,places.displayName,places.location,places.businessStatus,places.formattedAddress,places.primaryTypeDisplayName,places.primaryType",
    }
    url = "https://places.googleapis.com/v1/places:searchNearby"

    data = {
            "includedTypes": categories,
            "maxResultCount": max_results,
            "locationRestriction": {
                "circle": {
                    "center": {"latitude": lat, "longitude": lon},
                    "radius": radius,
                }
            },
        }
    
    r = requests.post(url, headers=headers, json=data)
    if r.status_code != 200:
        logging.warning(f"error at request: {r.request}")
        return

    data = r.json().get('places', [])
    if not data:
        return data

    if len(data) >= max_results:
        logging.warning(f'query at lon:{lon}, lat:{lat} returned 20+ results')
    
    for r in data:
        r['name'] = r['displayName']['text']
        r['geometry'] = Point(r['location']['longitude'], r['location']['latitude'])

    return data



In [4]:
def h3cell_to_shapely_polygon(cell:str):
    coords = h3.cell_to_boundary(cell)
    flipped = tuple(coord[::-1] for coord in coords)
    return Polygon(flipped)

## inputs

In [5]:
apikey = os.getenv("GOOGLE_API_KEY")
radius = 200
step = 60
buffer = 30
categories = ['cafe', 'bakery']
map_style = "open-street-map"

# Geospatial Analysis

## download network (roads)

In [6]:
bbox = [23.7576, 37.8524, 23.7729, 37.899]
g = ox.graph_from_bbox(bbox) # we can modify filters
nodes, links = ox.graph_to_gdfs(g)

## create the linestring geometry

In [12]:
road = links[links['name'] == 'Δημητρίου Γούναρη']
crs = road.estimate_utm_crs()
geom_proj = road.to_crs(crs).union_all()
geoms_proj = gpd.GeoDataFrame(geometry=[linemerge(geom_proj)]).explode()
geoms_proj['length'] = geoms_proj.length
geom_proj = geoms_proj.sort_values(['length'], ascending=False).iloc[0].geometry
road_buffered = road.to_crs(crs).buffer(buffer).to_crs(epsg=4326).union_all()

In [15]:
fp = './data/cafes.gpkg'
length = geom_proj.length

try:
    gdf = gpd.read_file(fp)
except DataSourceError:
    places = []
    transformer = Transformer.from_crs(crs, "EPSG:4326", always_xy=True)
    for distance in tqdm(np.arange(0, length, step)):
        p = shapely.line_interpolate_point(geom_proj, distance) # type:ignore
        p = transformer.transform(p.x, p.y)
        p = shapely.Point(p[0], p[1])
        r = fetch_google_places(categories, p.x, p.y, radius, apikey)
        if r is not None:
            places.extend(r)

    gdf = gpd.GeoDataFrame(places, crs='epsg:4326').drop_duplicates(subset=['id'])
    gdf['inbuffer'] = gdf.intersects(road_buffered) # keep cafes up to 30 meters from the road (inside the buffer)
    gdf.to_file(fp)    

In [18]:
road

osmid    highway lanes               name  \
u           v          key                                                   
269431727   5613525029 0     259190379  secondary     2  Δημητρίου Γούναρη   
269431804   1127993262 0     259190379  secondary     2  Δημητρίου Γούναρη   
            269438987  0     259190379  secondary     2  Δημητρίου Γούναρη   
269432478   4370530165 0    1223427873  secondary     2  Δημητρίου Γούναρη   
            269438938  0     331804280  secondary     2  Δημητρίου Γούναρη   
...                                ...        ...   ...                ...   
9875151832  269435571  0     459349888  secondary     2  Δημητρίου Γούναρη   
9875151835  9875151832 0     459349888  secondary     2  Δημητρίου Γούναρη   
            269435572  0     459349888  secondary     2  Δημητρίου Γούναρη   
11138476961 269592684  0     169052716  secondary     2  Δημητρίου Γούναρη   
            269593878  0     169052716  secondary     2  Δημητρίου Γούναρη   

                            oneway reversed     length  \
u           v          key                               
269431727   5613525029 0     False     True  48.902729   
269431804   1127993262 0     False    False  43.561884   
            269438987  0     False     True  59.459336   
269432478   4370530165 0     False     True   6.358468   
            269438938  0     False    False  23.332866   
...                            ...      ...        ...   
9875151832  269435571  0     False    False  21.293923   
9875151835  9875151832 0     False    False  39.318258   
            269435572  0     False     True  32.808225   
11138476961 269592684  0     False    False   6.397749   
            269593878  0     False     True  79.061465   

                                                                     geometry  \
u           v          key                                                      
269431727   5613525029 0    LINESTRING (23.76116 37.89531, 23.7612 37.8949...   
269431804   1127993262 0     LINESTRING (23.76103 37.89391, 23.76111 37.8943)   
            269438987  0    LINESTRING (23.76103 37.89391, 23.76091 37.89339)   
269432478   4370530165 0    LINESTRING (23.76069 37.89249, 23.76069 37.89243)   
            269438938  0    LINESTRING (23.76069 37.89249, 23.76069 37.892...   
...                                                                       ...   
9875151832  269435571  0    LINESTRING (23.76321 37.88197, 23.76317 37.882...   
9875151835  9875151832 0    LINESTRING (23.76333 37.88163, 23.76321 37.88197)   
            269435572  0    LINESTRING (23.76333 37.88163, 23.76343 37.88135)   
11138476961 269592684  0    LINESTRING (23.76831 37.86503, 23.76829 37.86509)   
            269593878  0    LINESTRING (23.76831 37.86503, 23.76837 37.864...   

                           maxspeed junction access service width tunnel  
u           v          key                                                
269431727   5613525029 0        NaN      NaN    NaN     NaN   NaN    NaN  
269431804   1127993262 0        NaN      NaN    NaN     NaN   NaN    NaN  
            269438987  0        NaN      NaN    NaN     NaN   NaN    NaN  
269432478   4370530165 0        NaN      NaN    NaN     NaN   NaN    NaN  
            269438938  0        NaN      NaN    NaN     NaN   NaN    NaN  
...                             ...      ...    ...     ...   ...    ...  
9875151832  269435571  0        NaN      NaN    NaN     NaN   NaN    NaN  
9875151835  9875151832 0        NaN      NaN    NaN     NaN   NaN    NaN  
            269435572  0        NaN      NaN    NaN     NaN   NaN    NaN  
11138476961 269592684  0        NaN      NaN    NaN     NaN   NaN    NaN  
            269593878  0        NaN      NaN    NaN     NaN   NaN    NaN  

[199 rows x 14 columns]

In [16]:
gdf['inbuffer'] = gdf.intersects(road_buffered) # keep cafes up to 30 meters from the road (inside the buffer)
cafes = gdf[gdf['inbuffer']]
cafes.head(2)

,id,formattedAddress,location,businessStatus,displayName,primaryTypeDisplayName,primaryType,name,inbuffer,geometry
1,ChIJgcEWYU6_oRQR-cd2bf6di68,"Attikis 1, Glifada 165 62, Greece","{'latitude': 37.894874, 'longitude': 23.761325...",OPERATIONAL,"{'text': 'coffee berry - Άνω Γλυφάδα', 'langua...","{'text': 'Coffee Shop', 'languageCode': 'en-US'}",coffee_shop,coffee berry - Άνω Γλυφάδα,True,POINT (23.76133 37.89487)
2,ChIJgSMfxKu_oRQRDDje9yAhZfM,"Dim. Gounari 24, Elliniko 167 77, Greece","{'latitude': 37.893558399999996, 'longitude': ...",OPERATIONAL,"{'text': 'ROSEWOOD COFFEE HOUSE', 'languageCod...","{'text': 'Coffee Shop', 'languageCode': 'en-US'}",coffee_shop,ROSEWOOD COFFEE HOUSE,True,POINT (23.76089 37.89356)


## stats / visuals

In [17]:
fig = px.scatter_map(gdf, lat=gdf.geometry.y, lon=gdf.geometry.x, color='inbuffer', text='name', hover_name="name", zoom=12, map_style=map_style)
fig.show()

In [19]:
fig = px.density_map(cafes, lat=cafes.geometry.y, lon=cafes.geometry.x, radius=12, zoom=12, map_style=map_style, width=600)
fig.show()

In [ ]:
f'One cafe every: {length / cafes.shape[0]:.0f}m'

'One cafe every: 125m'

### Distance between cafes (distance matrix)

In [22]:
gdf = cafes.to_crs(crs) # project the coordinates

dists = []
for i, g in gdf.iterrows():
    geom = g.geometry
    d = gdf.distance(geom).tolist()
    dists.append(d)

dists = pd.DataFrame(dists, index=gdf.index, columns=gdf.index).round(2)
dists

,1,2,3,8,9,11,14,15,16,18,...,39,40,44,45,46,47,48,50,51,52
1,0.00,150.93,32.59,216.53,312.53,396.85,747.69,822.68,1089.75,1100.24,...,2350.19,2361.13,2474.07,2756.83,2731.11,2892.25,2936.57,3145.88,3191.30,3234.03
2,150.93,0.00,151.69,67.89,164.76,250.26,602.27,678.12,948.32,958.73,...,2209.48,2221.41,2333.40,2616.98,2590.97,2753.75,2799.14,3009.77,3055.33,3098.25
3,32.59,151.69,0.00,219.36,316.27,401.58,753.50,829.11,1098.18,1108.62,...,2359.21,2370.68,2483.12,2766.34,2740.46,2902.47,2947.32,3157.26,3202.74,3245.56
8,216.53,67.89,219.36,0.00,96.92,182.37,534.38,610.23,880.61,891.00,...,2141.76,2153.78,2265.68,2549.33,2523.29,2686.25,2731.79,2942.60,2988.17,3031.13
9,312.53,164.76,316.27,96.92,0.00,85.59,437.55,513.51,784.56,794.92,...,2045.59,2057.86,2169.51,2453.34,2427.22,2590.60,2636.45,2847.64,2893.25,2936.26
11,396.85,250.26,401.58,182.37,85.59,0.00,352.01,427.92,699.14,709.49,...,1960.10,1972.47,2084.01,2367.91,2341.76,2505.33,2551.33,2762.73,2808.37,2851.40
14,747.69,602.27,753.50,534.38,437.55,352.01,0.00,76.82,351.40,361.45,...,1609.83,1623.08,1733.71,2018.13,1991.75,2156.71,2203.80,2416.48,2462.24,2505.46
15,822.68,678.12,829.11,610.23,513.51,427.92,76.82,0.00,274.70,284.71,...,1533.16,1546.31,1657.04,1941.40,1915.05,2079.91,2126.98,2339.68,2385.45,2428.67
16,1089.75,948.32,1098.18,880.61,784.56,699.14,351.40,274.70,0.00,10.57,...,1261.16,1273.32,1385.08,1668.78,1642.69,1806.36,1852.89,2065.22,2110.97,2154.16
18,1100.24,958.73,1108.62,891.00,794.92,709.49,361.45,284.71,10.57,0.00,...,1250.76,1262.99,1374.68,1658.42,1632.31,1796.08,1842.68,2055.09,2100.84,2144.04


In [23]:
nearests = gdf.sjoin_nearest(gdf, distance_col='sdist', exclusive=True)[['id_left', 'id_right', 'sdist']]
nearests
nearests['sdist'].mean()

np.float64(50.49189035272861)

In [24]:
nearests

,id_left,id_right,sdist
1,ChIJgcEWYU6_oRQR-cd2bf6di68,ChIJ78DvEB2-oRQRJRLxgttGeas,32.592452
2,ChIJgSMfxKu_oRQRDDje9yAhZfM,ChIJS8Wq1sS_oRQRV63ytQ_llsc,67.891246
3,ChIJ78DvEB2-oRQRJRLxgttGeas,ChIJgcEWYU6_oRQR-cd2bf6di68,32.592452
8,ChIJS8Wq1sS_oRQRV63ytQ_llsc,ChIJgSMfxKu_oRQRDDje9yAhZfM,67.891246
9,ChIJWd9MyR2-oRQRnUIGI3XRrl0,ChIJo9Fudx6-oRQRSYYEmtYc1R8,85.594782
11,ChIJo9Fudx6-oRQRSYYEmtYc1R8,ChIJWd9MyR2-oRQRnUIGI3XRrl0,85.594782
14,ChIJe3_1uh--oRQRlMU-4IznfWs,ChIJW9F_ix--oRQRpUygAqwjuy8,76.820468
15,ChIJW9F_ix--oRQRpUygAqwjuy8,ChIJe3_1uh--oRQRlMU-4IznfWs,76.820468
16,ChIJ31YIbCC-oRQR94INzAZyomg,ChIJ9aCCNMO_oRQRa27Yn0QAuq0,10.573095
18,ChIJ9aCCNMO_oRQRa27Yn0QAuq0,ChIJ31YIbCC-oRQR94INzAZyomg,10.573095


In [ ]:
cafes[cafes.id.isin(['ChIJgcEWYU6_oRQR-cd2bf6di68', 'ChIJ78DvEB2-oRQRJRLxgttGeas'])]

In [ ]:
nearests

### spatial grouping (h3)

In [27]:
box = shapely.box(*bbox)
coords = [list(box.exterior.coords)]
cells = h3.geo_to_cells(box, res=10)
cells = [{'h3id': cell, 'geometry': h3cell_to_shapely_polygon(cell)} for cell in cells]
cells = gpd.GeoDataFrame(cells, crs='epsg:4326')
cells.head(2)
px.choropleth_map(cells, geojson=cells.geometry, locations=cells.index,
                  center={'lat': box.centroid.y, 'lon': box.centroid.x}, zoom=11.5,
                  map_style=map_style)

In [26]:
counts = gpd.sjoin(cells, cafes, predicate='contains').groupby(['h3id', 'geometry'], as_index=False)['index_right'].count()
counts = counts.rename(columns={'index_right':'count'})
counts = gpd.GeoDataFrame(counts) # groupby casts to pd.DataFrame
px.choropleth_map(counts, geojson=counts.geometry, locations=counts.index,
                  color='count', color_continuous_scale=px.colors.sequential.Reds,
                  center={'lat': box.centroid.y, 'lon': box.centroid.x}, zoom=11.5,
                  title='Concentration of cafes (H3 cells)', map_style=map_style)